In [1]:
import numpy as np
import pandas as pd

In [78]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None)
movies.columns = ['movieid','name','genre']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [79]:
movies.head()

,movieid,name,genre
0,1,Toy Story,"Animation,Children,Comedy"
1,2,Jumanji,"Adventure,Children,Fantasy"
2,3,Grumpier Old Men,"Comedy,Romance"
3,4,Waiting to Exhale,"Comedy,Drama"
4,5,Father of the Bride Part II,Comedy


In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None)
ratings.columns = ['userid','movieid','rating','time']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
ratings.head()

,userid,movieid,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings = ratings.drop(['userid','time'], axis =1)

In [7]:
ratings.sort_values(by=['movieid'], inplace=True)

In [8]:
ratings.head()

,movieid,rating
427702,1,4
1966,1,4
683688,1,5
596207,1,4
465902,1,5


In [9]:
m_ids = []
rates = []
for i in range(len(ratings['movieid'])):
    sigma = ratings[ratings['movieid'] == i].mean()
    m_id = round(sigma[0])
    rate = round(sigma[1])
    m_ids.append(m_id)
    rates.append(rate)
print(m_ids,rates)
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
ratings = {
    "movieid":m_ids,
    "rating":rates
}
ratings = pd.DataFrame(ratings)

In [11]:
ratings = ratings.dropna()

In [58]:
ratings.head()

,movieid,rating
1,1.0,4.0
2,2.0,3.0
3,3.0,3.0
4,4.0,3.0
5,5.0,3.0


In [80]:
movies = movies.merge(ratings , on = 'movieid')

In [81]:
movies.head()

,movieid,name,genre,rating
0,1,Toy Story,"Animation,Children,Comedy",4.0
1,2,Jumanji,"Adventure,Children,Fantasy",3.0
2,3,Grumpier Old Men,"Comedy,Romance",3.0
3,4,Waiting to Exhale,"Comedy,Drama",3.0
4,5,Father of the Bride Part II,Comedy,3.0


In [82]:
user_movies = ["She's the One","Time to Kill","American Buffalo","Rendezvous in Paris (Rendez-vous de Paris","Alaska","Fled",
               "Kazaam","Bewegte Mann","Magic Hunter","Larger Than Life","Boy Called Hate","Power 98","Two Deaths",
               "Very Brady Sequel","Stefano Quantestorie","Death in the Garden (Mort en ce jardin","Crude Oasis" ]

In [183]:
genres = []
for i in range(len(movies['genre'])):
    genre = movies['genre'][i].split(',')
    for j in range(len(genre)):
        genres.append(genre[j])
genres = np.unique(genres)

In [190]:
genres

array(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Horror', 'Musical', 'Mystery',
       'Noir', 'Romance', 'SciFi', 'Thriller', 'War', 'Western'],
      dtype='<U11')

In [186]:
genres = genres[2:]

In [191]:
genres.shape

(18,)

In [87]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    score = len(s1.intersection(s2)) / len(s1.union(s2))
    return score

In [102]:
len(movies['genre'])

3706

In [241]:
def scores(user_movies):
    Scores=[]
    for i in range(len(genres)):
        m_list = []
        for j in range(len(movies['genre'])):
            genre = movies['genre'][j].split(',')
            for k in range(len(genre)):
    #             print(genre[k])
    #             print(genres[i])
                if genre[k] == genres[i]:
                    m_list.append(movies['name'][j])
    #         print(m_list)
        score = jaccard_similarity(user_movies,m_list)
        Scores.append(score)         
    genre_index = np.argmax(Scores)
    max_genre = genres[genre_index]
    return max_genre, Scores

In [243]:
max_genre, Scores = scores(user_movies)

In [220]:
max_genre

'Romance'

In [221]:
def genre(max_genre):
    rec_mov = []
    mov_rate = []
    for j in range(len(movies['genre'])):
        genre = movies['genre'][j].split(',')
#         print(genre)
        for k in range(len(genre)):
#             print(genre[k])
            if genre[k] == max_genre:
#                 print(genre[k])
                rec_mov.append(movies['name'][j])
                mov_rate.append(movies['rating'][j])
    mov_rates = np.array(np.argsort(mov_rate))
    rate_index = np.array(mov_rates[-10:])
    movie_names = []
    for i in range(len(rate_index)):
        mov_index = rate_index[-i]
        movie_names.append(rec_mov[mov_index])
    return movie_names

In [237]:
def result(max_genre):
    genre_score = []
    for i in range(18):
        genre_score.append(genres[i])
        genre_score.append(Scores[i])
    print("Scores :- ")
    for i in range(0,len(genre_score),2):
        print("   ",genre_score[i]," = ",genre_score[i+1])
    print("Suggested Movies :-")
    for i in range(10):
        print("   ",i+1,".",genre(max_genre)[i])    

In [238]:
result(max_genre)

Scores :- 
    Action  =  0.0
    Adventure  =  0.0
    Animation  =  0.0
    Children  =  0.0
    Comedy  =  0.0026041666666666665
    Crime  =  0.0
    Documentary  =  0.0
    Drama  =  0.0020408163265306124
    Fantasy  =  0.0
    Horror  =  0.0
    Musical  =  0.0
    Mystery  =  0.0
    Noir  =  0.0
    Romance  =  0.0045045045045045045
    SciFi  =  0.0
    Thriller  =  0.0
    War  =  0.0
    Western  =  0.0
Suggested Movies :-
    1 . My Fair Lady  
    2 . I Love You
    3 . Ever After: A Cinderella Story  
    4 . Rich and Strange  
    5 . Déjà Vu  
    6 . American in Paris
    7 . Return to Paradise  
    8 . Breakfast at Tiffany's  
    9 . Charade  
    10 . Casablanca  


In [193]:
genre_score = []
for i in range(18):
    genre_score.append(genres[i])
    genre_score.append(Scores[i])
# print(genre_score)
# len(genre_score)

['Action', 0.0, 'Adventure', 0.0, 'Animation', 0.0, 'Children', 0.0, 'Comedy', 0.0026041666666666665, 'Crime', 0.0, 'Documentary', 0.0, 'Drama', 0.0020408163265306124, 'Fantasy', 0.0, 'Horror', 0.0, 'Musical', 0.0, 'Mystery', 0.0, 'Noir', 0.0, 'Romance', 0.0045045045045045045, 'SciFi', 0.0, 'Thriller', 0.0, 'War', 0.0, 'Western', 0.0]


36

In [206]:
print("Scores :- ")
for i in range(0,len(genre_score),2):
    print(genre_score[i]," = ",genre_score[i+1])

Scores : 
Action  =  0.0
Adventure  =  0.0
Animation  =  0.0
Children  =  0.0
Comedy  =  0.0026041666666666665
Crime  =  0.0
Documentary  =  0.0
Drama  =  0.0020408163265306124
Fantasy  =  0.0
Horror  =  0.0
Musical  =  0.0
Mystery  =  0.0
Noir  =  0.0
Romance  =  0.0045045045045045045
SciFi  =  0.0
Thriller  =  0.0
War  =  0.0
Western  =  0.0
